# TensorFlow 2.0

In [ ]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [ ]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# 압축 풀기
!unzip -qq /content/drive/"My Drive"/"[AOP:딥러닝 인공지능]"/TEST/"1. [Tensorflow 2.0 과 Pytorch]"/"3. 이미지 분석으로 배우는 tensorflow 2.0과 Pytorch"/"3. dataset"/cifar.zip


In [ ]:
os.listdir('/content/cifar/')

['labels.txt', 'test', 'train']

In [ ]:
train_paths = glob('/content/cifar/train/*.png')[:100]
test_paths = glob('/content/cifar/test/*.png')[:100]

In [ ]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [ ]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [ ]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [ ]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [ ]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [ ]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
3/3 [==============================] - 1s 232ms/step - loss: 2.3722 - accuracy: 0.1354 - val_loss: 2.2996 - val_accuracy: 0.1458
Epoch 2/5
3/3 [==============================] - 1s 206ms/step - loss: 2.2721 - accuracy: 0.2059 - val_loss: 2.2999 - val_accuracy: 0.1458
Epoch 3/5
3/3 [==============================] - 1s 179ms/step - loss: 2.2381 - accuracy: 0.2059 - val_loss: 2.3098 - val_accuracy: 0.1458
Epoch 4/5
3/3 [==============================] - 0s 150ms/step - loss: 2.4879 - accuracy: 0.2059 - val_loss: 2.3401 - val_accuracy: 0.1458
Epoch 5/5
3/3 [==============================] - 1s 189ms/step - loss: 2.3205 - accuracy: 0.1912 - val_loss: 2.3129 - val_accuracy: 0.1458


# Load Checkpoint

# Saving Model

In [19]:
save_path = '/content/my_model.h5'

In [20]:
model.save(save_path, include_optimizer=True)

In [21]:
model = tf.keras.models.load_model('/content/my_model.h5')

# Saving Model - 2

In [22]:
model.save_weights('model_weights.h5')

In [23]:
with open('/content/model_architecture.json', 'w') as f:
  f.write(model.to_json())

In [24]:
from tensorflow.keras.models import model_from_json

In [25]:
with open('/content/model_architecture.json','r') as f:
  model = model_from_json(f.read())

In [26]:
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [27]:
import h5py

In [29]:
model_file = h5py.File('my_model.h5', 'r+')

In [30]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [31]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [32]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [33]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [35]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [36]:
weight

array([[[[-3.66097838e-02, -1.28953531e-01, -9.08701196e-02,
           8.07273984e-02, -8.17036703e-02, -1.21979348e-01,
          -9.62217376e-02, -5.15371896e-02,  1.10153005e-01,
           1.18653633e-01, -6.98712394e-02, -6.29650727e-02,
           9.60927829e-02, -1.12503812e-01, -8.87450650e-02,
          -1.34827808e-01,  1.17947064e-01,  8.72314796e-02,
          -1.06202021e-01,  4.08893079e-02,  2.41462397e-03,
           1.09146923e-01, -1.12968087e-01,  1.10036535e-02,
          -9.97726768e-02, -6.62183464e-02,  2.55011488e-02,
          -4.82166298e-02, -1.12670153e-01,  1.04116872e-02,
           1.20218433e-01,  7.18212053e-02],
         [ 4.93689589e-02,  3.44628356e-02,  1.29849717e-01,
           1.09441774e-02, -1.64914597e-02,  2.06914116e-02,
          -6.30825683e-02, -1.17458932e-01,  5.98718859e-02,
          -9.07893926e-02,  6.41699973e-03, -9.46100205e-02,
           9.21126679e-02, -3.46894935e-02,  7.92289246e-03,
           9.50413849e-03,  1.06364273e-